# Báo cáo đồ án môn "Khoa học dữ liệu ứng dụng"

Nhóm 11:
1. 1612355 - Lê Kinh Luân - https://github.com/kluan98
2. 1712284 - Hoàng Gia Bảo - https://github.com/hgbao8799
3. 18120052 - Lê Hạnh Linh - https://github.com/linh0303052
4. 18120182 - Lê Hồng Huy - https://github.com/lehonghuy2000
5. 1712371 - Nguyễn Văn Dưng - https://github.com/dungdev1

Link thùng chứa Github của nhóm: https://github.com/linh0303052/AplliedDSGroup11

## Mô tả bài toán

Bài toán **Santander Customer Satisfaction** được tổ chức bởi Ngân hàng Santander có thị trường là Đông Bắc Hoa Kỳ và là công ty con của tập đoàn Santander ở Tây Ban Nha.

**Link:** https://www.kaggle.com/c/santander-customer-satisfaction/overview

**Vấn đề:** Phân tích sự hài lòng của khách hàng là phép đo chìa khóa của thành công bởi vì các khách hàng không hài lòng sẽ có ít cơ hội mua lại lần tiếp theo và quan trọng hơn là họ sẽ nói lên sự không hài lòng của họ trên các nền tảng xã hội trước khi rời đi. Ngân hàng Santander mong muốn phát hiện sớm các khách hàng không hài lòng và có các giải pháp chủ động để cải thiện trải nghiệm, thái độ của khách hàng trước khi quá muộn.

**Input:** Dữ liệu gồm 370 thuộc tính được anonymized

**Output:** Dự đoán đây có phải là khách hàng không hài lòng với kết quả là 1 nếu *không hài lòng* hoặc 0 nếu *hài lòng*

## Giải quyết bài toán

Trong đồ án này, nhóm tìm hiểu và cài đặt lại phương pháp của nhóm **#1 Leustagos** đặt hạng 3 trên Private Leaderboard

**Github:** https://github.com/diefimov/santander_2016

**Mô tả:** Giải pháp của tác giả chủ yếu dựa trên kết hợp (ensemble) của các mô hình khác nhau vì dataset gồm rất nhiều thuộc tính nên việc dùng ensemble để tránh tình trạng overfit. Các tác giả sử dụng khá đa dạng các mô hình có tính chất làm việc tốt trên dataset bị ẩn danh hóa và dạng bảng như Random Forest, Adaboost Classifier, XGBOOST, Neural Network. Mỗi nhóm mô hình có cách tiền xử lý dữ liệu khác nhau dành cho mỗi mô hình thay vì chỉ tiền xử lý một lần đầu và dùng cho tất cả mô hình. Đây là lí do nhóm chọn giải pháp của nhóm tác giả này trong số các giải pháp top đầu.

Giải pháp này là một ensemble của 5 nhóm mô hình lớn, được tiền xử lý và chạy độc lập với nhau. Đặc điểm này cũng quy định cấu trúc của phần tiếp theo: mỗi nhóm mô hình sẽ được chia thành 2 phần lớn là phần *tiền xử lý dữ liệu* và phần *mô hình*.

### Import các thư viện cần thiết

In [26]:
import numpy as np
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.preprocessing import normalize
from sklearn.model_selection import StratifiedKFold
from scipy.stats.stats import pearsonr
from sklearn.decomposition import PCA
from sklearn.preprocessing import normalize
from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import StandardScaler
from sklearn.manifold import TSNE

### Tải dữ liệu

In [39]:
train = pd.read_csv('./data/input/train.csv')
train.shape

(76020, 371)

In [40]:
test = pd.read_csv('./data/input/test.csv')
test.shape

(75818, 370)

In [41]:
INPUT_PATH = './data/input/'
OUTPUT_PATH = './data/output/'

### Nhóm mô hình Dmitry Efimov

#### Tiền xử lý dữ liệu

In [42]:
#xử lý các giá trị đặc biệt, thay thế nó bằng giá trị NA (-999.0)
train.loc[(train['var38']>117310.979) & (train['var38']<117310.98), 'var38'] = -999.0
test.loc[(test['var38']>117310.979) & (test['var38']<117310.98), 'var38'] = -999.0

train.loc[train['var3']==-999999, 'var3'] = -999.0
test.loc[test['var3']==-999999, 'var3'] = -999.0

for f in ['imp_op_var40_comer_ult1', 'imp_op_var40_efect_ult3', 'imp_op_var41_comer_ult3', 'imp_sal_var16_ult1']:
    train.loc[train[f]==0.0, f] = -999.0
    test.loc[test[f]==0.0, f] = -999.0

In [43]:
#Loại bỏ các thuộc tính chỉ có 1 giá trị
flist = [x for x in train.columns if not x in ['ID','TARGET']]
for f in flist:
    if len(np.unique(train[f]))<2:
        train.drop(f, axis=1, inplace=True)
        test.drop(f, axis=1, inplace=True)

In [44]:
#Loại bỏ var6 vì nó trùng với var29
flist = [x for x in train.columns if not x in ['ID','TARGET']]            
train.drop([x for x in flist if 'var6' in x], axis=1, inplace=True)
test.drop([x for x in flist if 'var6' in x], axis=1, inplace=True)

#Loại bỏ các thuộc tính có chứa _0 vì nó bị trùng với cột có chứa _1 theo ngay sau
flist = [x for x in train.columns if not x in ['ID','TARGET']]        
flist_remove = []
for i in range(len(flist)-1):
    v = train[flist[i]].values
    for j in range(i+1, len(flist)):
        if np.array_equal(v, train[flist[j]].values):
            if '_0' in flist[j]:
                flist_remove.append(flist[j])
            elif  '_0' in flist[i]:
                flist_remove.append(flist[i])
train.drop(flist_remove, axis=1, inplace=True)
test.drop(flist_remove, axis=1, inplace=True)

#Loại bỏ các cột bị trùng khác
flist_remove = ['saldo_medio_var13_medio_ult1', 'delta_imp_reemb_var13_1y3', 'delta_imp_reemb_var17_1y3', 
                   'delta_imp_reemb_var33_1y3', 'delta_imp_trasp_var17_in_1y3', 'delta_imp_trasp_var17_out_1y3',
                   'delta_imp_trasp_var33_in_1y3', 'delta_imp_trasp_var33_out_1y3']
train.drop(flist_remove, axis=1, inplace=True)
test.drop(flist_remove, axis=1, inplace=True)

In [ ]:
#Chuẩn hóa các giá trị thuộc tính
def normalize_features(train, test):
    flist = [x for x in train.columns if not x in ['ID','TARGET']]
    for f in flist:
        if train[f].max() == 9999999999.0:
            fmax = train.loc[train[f]<9999999999.0, f].max()
            train.loc[train[f]==9999999999.0, f] = fmax + 1

        if len(train.loc[train[f]<0, f].value_counts()) == 1:
            train.loc[train[f]<0, f] = -1.0
            test.loc[test[f]<0, f] = -1.0
            fmax = max(np.max(train[f]), np.max(test[f]))
            if fmax > 0:
                train.loc[train[f]>0, f] = 1.0*train.loc[train[f]>0, f]/fmax
                test.loc[test[f]>0, f] = 1.0*test.loc[test[f]>0, f]/fmax

        if len(train.loc[train[f]<0, f]) == 0:
            fmax = max(np.max(train[f]), np.max(test[f]))
            if fmax > 0:
                train.loc[train[f]>0, f] = 1.0*train.loc[train[f]>0, f]/fmax
                test.loc[test[f]>0, f] = 1.0*test.loc[test[f]>0, f]/fmax

        if len(train.loc[train[f]<0, f].value_counts()) > 1:
            fmax = max(np.max(train[f]), np.max(test[f]))
            if fmax > 0:
                train[f] = 1.0*train[f]/fmax
                test[f] = 1.0*test[f]/fmax

    return train, test

#### Feature Engineering

In [ ]:
#thuộc tính t_SNE
np.random.seed(12324)
train_tsne, test_tsne = add_features(train, test, ['SumZeros'])

flist = [x for x in train_tsne.columns if not x in ['ID','TARGET']]

X = train_tsne[flist].append(test[flist], ignore_index=True).values.astype('float64')
svd = TruncatedSVD(n_components=30)
X_svd = svd.fit_transform(X)
X_scaled = StandardScaler().fit_transform(X_svd)
feats_tsne = TSNE(n_components=2, random_state=0).fit_transform(X_scaled)
feats_tsne = pd.DataFrame(feats_tsne, columns=['tsne1', 'tsne2'])
feats_tsne['ID'] = train_tsne[['ID']].append(test_tsne[['ID']], ignore_index=True)['ID'].values
train_tsne = pd.merge(train_tsne, feats_tsne, on='ID', how='left')
test_tsne = pd.merge(test_tsne, feats_tsne, on='ID', how='left')

feat = train[['ID', 'tsne1', 'tsne2']].append(test[['ID', 'tsne1', 'tsne2']], ignore_index=True)
feat.to_csv(OUTPUT_PATH + 'tsne_feats.csv', index=False)

In [ ]:
#thuộc tính PCA
train_pca, test_pca = add_features(train, test, ['SumZeros'])

flist = [x for x in train_pca.columns if not x in ['ID','TARGET']]

pca = PCA(n_components=2)
x_train_projected = pca.fit_transform(normalize(train_pca[flist], axis=0))
x_test_projected = pca.transform(normalize(test_pca[flist], axis=0))
train_pca.insert(1, 'PCAOne', x_train_projected[:, 0])
train_pca.insert(1, 'PCATwo', x_train_projected[:, 1])
test_pca.insert(1, 'PCAOne', x_test_projected[:, 0])
test_pca.insert(1, 'PCATwo', x_test_projected[:, 1])
pca_feats = train_pca[['ID', 'PCAOne', 'PCATwo']].append(test_pca[['ID', 'PCAOne', 'PCATwo']], ignore_index=True)
pca_feats.to_csv(OUTPUT_PATH + 'dmitry_pca_feats.csv')

In [ ]:
#thuộc tính k-means
train_k, test_k = add_features(train_k, test, ['SumZeros'])
train_k, test_k = normalize_features(train_k, test)

flist = [x for x in train_k.columns if not x in ['ID','TARGET']]

flist_kmeans = []
for ncl in range(2,11):
    cls = KMeans(n_clusters=ncl)
    cls.fit_predict(train_k[flist].values)
    train_k['kmeans_cluster'+str(ncl)] = cls.predict(train_k[flist].values)
    test_k['kmeans_cluster'+str(ncl)] = cls.predict(test_k[flist].values)
    flist_kmeans.append('kmeans_cluster'+str(ncl))

train[['ID']+flist_kmeans].append(test[['ID']+flist_kmeans], ignore_index=True).to_csv(OUTPUT_PATH + 'kmeans_feats.csv', index=False)

In [14]:
#LL = (30.yTB + yG)/(30 + |G|)
def add_likelihood_feature(fname, train_likeli, test_likeli, flist):
    tt_likeli = pd.DataFrame()
    np.random.seed(1232345)
    #Chia fold để tính toán các giá trị
    #Tập test được điền theo tập train
    skf = StratifiedKFold(train_likeli['TARGET'].values, n_folds=5, shuffle=True, random_state=21387)
    for train_index, test_index in skf:
        ids = train_likeli['ID'].values[train_index]
        train_fold = train_likeli.loc[train_likeli['ID'].isin(ids)].copy()
        test_fold = train_likeli.loc[~train_likeli['ID'].isin(ids)].copy()
        global_avg = np.mean(train_fold['TARGET'].values)
        feats_likeli = train_fold.groupby(fname)['TARGET'].agg({'sum': np.sum, 'count': len}).reset_index()
        feats_likeli[fname + '_likeli'] = (feats_likeli['sum'] + 30.0*global_avg)/(feats_likeli['count']+30.0)
        test_fold = pd.merge(test_fold, feats_likeli[[fname, fname + '_likeli']], on=fname, how='left')
        test_fold[fname + '_likeli'] = test_fold[fname + '_likeli'].fillna(global_avg)
        tt_likeli = tt_likeli.append(test_fold[['ID', fname + '_likeli']], ignore_index=True)
    train_likeli = pd.merge(train_likeli, tt_likeli, on='ID', how='left')
    
    global_avg = np.mean(train_likeli['TARGET'].values)
    feats_likeli = train_likeli.groupby(fname)['TARGET'].agg({'sum': np.sum, 'count': len}).reset_index()
    feats_likeli[fname + '_likeli'] = (feats_likeli['sum'] + 30.0*global_avg)/(feats_likeli['count']+30.0)
    test_likeli = pd.merge(test_likeli, feats_likeli[[fname, fname + '_likeli']], on=fname, how='left')
    test_likeli[fname + '_likeli'] = test_likeli[fname + '_likeli'].fillna(global_avg)
    return train_likeli, test_likeli, flist + [fname + '_likeli']

In [12]:
def add_features(train, test, features):
    flist = [x for x in train.columns if not x in ['ID','TARGET']]
    if 'SumZeros' in features:
        train.insert(1, 'SumZeros', (train[flist] == 0).astype(int).sum(axis=1))
        test.insert(1, 'SumZeros', (test[flist] == 0).astype(int).sum(axis=1))
    flist = [x for x in train.columns if not x in ['ID','TARGET']]

    if 'tsne' in features:
        tsne_feats = pd.read_csv(OUTPUT_PATH + 'features/tsne_feats.csv')
        train = pd.merge(train, tsne_feats, on='ID', how='left')
        test = pd.merge(test, tsne_feats, on='ID', how='left')

    if 'pca' in features:
        pca_feats = pd.read_csv(OUTPUT_PATH + 'features/dmitry_pca_feats.csv')
        train = pd.merge(train, pca_feats, on='ID', how='left')
        test = pd.merge(test, pca_feats, on='ID', how='left')

    if 'kmeans' in features:
        kmeans_feats = pd.read_csv(OUTPUT_PATH + 'features/kmeans_feats.csv')
        train = pd.merge(train, kmeans_feats, on='ID', how='left')
        test = pd.merge(test, kmeans_feats, on='ID', how='left')

    return train, test

#### Mô hình

#####  FTRL2 (Follow the Regularized Leader)
Sử dụng dữ liệu gốc, Sumzeros và Likelihood

#####  Regularized Greedy Forest 3
Sử dụng dữ liệu gốc, Sumzeros, PCA và Likelihood

#####  Regularized Greedy Forest 5
Sử dụng dữ liệu gốc, Sumzeros, tSNE và Likelihood

#####  Regularized Greedy Forest 6
Sử dụng dữ liệu gốc, Sumzeros, K-means và Likelihood

#####  Adaboost classifier
Sử dụng dữ liệu gốc, Sumzeros, PCA và Likelihood

#####  XGBOOST
Sử dụng dữ liệu gốc, Sumzeros, PCA và Likelihood

### Nhóm mô hình Ikki Tanaka

#### Tiền xử lý dữ liệu

#### Mô hình

### Nhóm mô hình Darius Barusauskas

#### Tiền xử lý dữ liệu

#### Mô hình

### Nhóm mô hình Marios Michailidis

#### Tiền xử lý dữ liệu

#### Mô hình

### Nhóm mô hình Mathias Muller

#### Tiền xử lý dữ liệu

#### Mô hình

## Tài liệu tham khảo